In [1]:
import duckdb
import timeit

# DuckDB에 연결
conn = duckdb.connect(':memory:')

# 시간 측정 시작
start_time = timeit.default_timer()

# Parquet 파일을 DuckDB 테이블로 로드
conn.execute("CREATE TABLE table1 AS SELECT * FROM parquet_scan('/home/theo/dataset/test_data.parquet')")
conn.execute("CREATE TABLE table2 AS SELECT * FROM parquet_scan('/home/theo/dataset/train_data.parquet')")

# 두 테이블을 병합하고 distinct 작업 수행
result = conn.execute("SELECT DISTINCT * FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2)")

# 시간 측정 종료 및 소요 시간 출력
end_time = timeit.default_timer()
print("DuckDB 소요 시간:", end_time - start_time)


DuckDB 소요 시간: 2651.045915929


In [1]:
# distinct 적용 전
rows_before = conn.execute("SELECT COUNT(*) FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2)").fetchone()[0]
#unique_values_before = conn.execute("SELECT COUNT(DISTINCT COLUMNS(*)) FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2)").fetchone()[0]
#
# distinct 적용 후
rows_after = conn.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2))").fetchone()[0]
#unique_values_after = conn.execute("SELECT COUNT(DISTINCT COLUMNS(*)) FROM (SELECT DISTINCT * FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2))").fetchone()[0]

print("Distinct 적용 전 행 수:", rows_before)
#print("Distinct 적용 전 고유한 값 수:", unique_values_before)
print("Distinct 적용 후 행 수:", rows_after)
#print("Distinct 적용 후 고유한 값 수:", unique_values_after)


NameError: name 'conn' is not defined

In [ ]:
import duckdb
import pyarrow.parquet as pq
import timeit

# duckdb 연결
con = duckdb.connect(database=':memory:', read_only=False)

# duckdb 테이블 생성
con.execute("CREATE TABLE geoparquet (address VARCHAR, boundary DOUBLE)")

# 모든 GeoParquet 파일을 읽어서 duckdb에 데이터 로드
def load_data():
    for i in range(1, 18):
        file_path = f"path/to/geoparquet{i}.parquet"
        table = pq.read_table(file_path)
        df = table.to_pandas()
        con.register(f"geoparquet_temp{i}", df)
        con.execute(f"INSERT INTO geoparquet SELECT * FROM geoparquet_temp{i}")

# 중복 제거 쿼리 실행
def run_query():
    query = """
    SELECT DISTINCT address, boundary FROM geoparquet
    """
    result = con.execute(query)
    for row in result:
        print(row)

# 시간 측정
load_time = timeit.timeit(load_data, number=1)
query_time = timeit.timeit(run_query, number=1)

# 결과 출력
print(f"데이터 로드 시간: {load_time} 초")
print(f"중복 제거 쿼리 실행 시간: {query_time} 초")


In [ ]:
import duckdb
import timeit

# DuckDB에 연결
conn = duckdb.connect(':memory:')

# 시간 측정 시작
start_time = timeit.default_timer()

# Parquet 파일을 DuckDB 테이블로 로드
conn.execute("CREATE TABLE table1 AS SELECT * FROM parquet_scan('/home/theo/dataset/test_data.parquet')")
conn.execute("CREATE TABLE table2 AS SELECT * FROM parquet_scan('/home/theo/dataset/train_data.parquet')")

# 두 테이블을 병합하고 distinct 작업 수행
result = conn.execute("SELECT DISTINCT * FROM (SELECT * FROM table1 UNION ALL SELECT * FROM table2)")

# 시간 측정 종료 및 소요 시간 출력
end_time = timeit.default_timer()
print("DuckDB 소요 시간:", end_time - start_time)
